## Opcode computer based on classes

Need to modify existing opcode computer: 

First, you'll need to add two new instructions:

* Opcode 3 takes a single integer as input and saves it to the position given by its only parameter. For example, the instruction 3,50 would take an input value and store it at address 50.
* Opcode 4 outputs the value of its only parameter. For example, the instruction 4,50 would output the value at address 50. 

Programs that use these instructions will come with documentation that explains what should be connected to the input and output. The program 3,0,4,0,99 outputs whatever it gets as input, then halts.

Functions to load and parse data:

In [10]:
class Computer():
    
    def __init():
        pass
    
    def load_data(self, path):
        with open(path) as f:
            program = f.read()
        self.program = program
        return self.program

    def parse_data(self): 
        """Function to split string and convert to int"""

        program = self.program.split(',')
        program = [int(value.rstrip('\n')) for value in program]
        self.program = program
        return self.program

In [12]:
computer = Computer()
computer.load_data('data/day_5.txt')
print(computer.parse_data())

[3, 225, 1, 225, 6, 6, 1100, 1, 238, 225, 104, 0, 1102, 9, 19, 225, 1, 136, 139, 224, 101, -17, 224, 224, 4, 224, 102, 8, 223, 223, 101, 6, 224, 224, 1, 223, 224, 223, 2, 218, 213, 224, 1001, 224, -4560, 224, 4, 224, 102, 8, 223, 223, 1001, 224, 4, 224, 1, 223, 224, 223, 1102, 25, 63, 224, 101, -1575, 224, 224, 4, 224, 102, 8, 223, 223, 1001, 224, 4, 224, 1, 223, 224, 223, 1102, 55, 31, 225, 1101, 38, 15, 225, 1001, 13, 88, 224, 1001, 224, -97, 224, 4, 224, 102, 8, 223, 223, 101, 5, 224, 224, 1, 224, 223, 223, 1002, 87, 88, 224, 101, -3344, 224, 224, 4, 224, 102, 8, 223, 223, 1001, 224, 7, 224, 1, 224, 223, 223, 1102, 39, 10, 225, 1102, 7, 70, 225, 1101, 19, 47, 224, 101, -66, 224, 224, 4, 224, 1002, 223, 8, 223, 1001, 224, 6, 224, 1, 224, 223, 223, 1102, 49, 72, 225, 102, 77, 166, 224, 101, -5544, 224, 224, 4, 224, 102, 8, 223, 223, 1001, 224, 4, 224, 1, 223, 224, 223, 101, 32, 83, 224, 101, -87, 224, 224, 4, 224, 102, 8, 223, 223, 1001, 224, 3, 224, 1, 224, 223, 223, 1101, 80, 5, 225

Now we need a program to parse each opcode, identifying the code and its parameters. How to do this? Account for the fact that initial zeros will be missing, so 0001 would be just 1.  

* Code 1 is 4 digits, from right to left: 2-digit code (01), parameter 1, parameter 2 (unstated parameter 3, always 0)
    * But could be just 1 digit, if all parameters are 0
* Code 2 is 4 digits, from right to left: 2-digit code (02), parameter 1, parameter 2 (unstated parameter 3, always 0)
    * But could be just 1 digit, if all parameters are 0
* Code 3 is 1 digit, just a '3'
* Code 4 is 1 digit, just a '4'
* Code 99 is 2 digits: 99

In [1]:
import intcode

In [50]:
def parse_opcode(opcode): 
    """Given an opcode int as part of a program, this function
    splits the string to identify: 
    1) The opcode number, 
    2) The modes of the first two parameters. Omitting parameter 3 because 
    it's a write operation and will always be 0 (position mode)."""
    
    full = str(opcode)
    if len(full) < 5: 
        full = full.rjust(5, "0")
    code = int(full[3:])
    mode_3, mode_2, mode_1 = int(full[0]), int(full[1]), int(full[2])
        
    return (code, mode_1, mode_2, mode_3)

# Test

print(parse_opcode('0102'))
print(parse_opcode('99'))
print(parse_opcode('3'))

(2, 1, 0, 0)
(99, 0, 0, 0)
(3, 0, 0, 0)


New approach: Create sub-function to process a single opcode. Keep the while loop to call it for each opcode, skipping ahead as determined by the pointer. 

In [61]:
def run_opcode(program, index, code, input_val = None):
    """Function to process a single opcode int located at the index specified, 
    given a full program and a tuple of code plus parameter modes."""
    
    p = program.copy()
    output = "No output"
    
    if code[0] in [1, 2]: 
        
        try: 
            param_1_dict = {0: p[p[index + 1]], # Use try except IndexError, as this can be out of range
                            1: p[index + 1]}
            param_2_dict = {0: p[p[index + 2]], # Same 
                            1: p[index + 2]}
        except IndexError:
            param_1_dict = {0: p[0], 
                            1: p[index + 1]}
            param_2_dict = {0: p[0], 
                            1: p[index + 2]}
            
    if code[0] == 1:
        p[p[index + 3]] = param_1_dict[code[1]] + param_2_dict[code[2]]
        index += 4
    elif code[0] == 2:
        p[p[index + 3]] = param_1_dict[code[1]] * param_2_dict[code[2]]
        index += 4
    elif code[0] == 3: 
        p[p[index + 1]] = input_val
        index += 2
    elif code[0] == 4:
        output = p[index + 1]
        index += 2
    elif code[0] == 99:
        output = "finished"
    else:
        raise Exception("Not a valid opcode")
        
    return (p, index, output) # In calling function, can retrieve tuple values by index

Run some tests on this function:

In [62]:
test3 = [1002, 4, 3, 4, 33]
print(test3)

[1002, 4, 3, 4, 33]


In [63]:
a = parse_opcode(test3[0])
print(a)

(2, 0, 1, 0)


In [64]:
b = run_opcode(program = test3, index = 0, code = a)
print(b)
type(b)

([1002, 4, 3, 4, 99], 4, 'No output')


tuple

In [65]:
test4 = [3, 2, 0]
c = parse_opcode(test4[0])
d = run_opcode(program = test4, index = 0, code = c, input_val = 5)
print(d)

([3, 2, 5], 2, 'No output')


Function to run the entire program:

In [73]:
def run_program_v1(program, input_val): 
    
    p = program.copy()
    pointer = 0
    finished = False
    output = []

    while pointer < len(p) and not finished: 
        code = parse_opcode(p[pointer]) 
        result = run_opcode(p, pointer, code, input_val)
        p = result[0]
        if result[2] == "finished":
            finished = True 
        else: 
            pointer = result[1]
            output.append((pointer, result[2]))
        
    return output, p

In [74]:
k = parse_opcode(test3[0])
print(k)

(2, 0, 1, 0)


In [75]:
print(run_opcode(test3, 0, k))

([1002, 4, 3, 4, 99], 4, 'No output')


In [76]:
x = run_program_v1(test3, input_val = None)
print(x)

([(4, 'No output')], [1002, 4, 3, 4, 99])


If necessary, see Ray's answer: https://github.com/raybuhr/adventofcode/commit/ce754779f006b572ddee38fbb14ef055ce2cd017

Now run it on the entire program:

In [77]:
z = run_program_v1(program, input_val = 1)
print(z)

([(2, 'No output'), (6, 'No output'), (10, 'No output'), (12, 0), (16, 'No output'), (20, 'No output'), (24, 'No output'), (26, 224), (30, 'No output'), (34, 'No output'), (38, 'No output'), (42, 'No output'), (46, 'No output'), (48, 224), (52, 'No output'), (56, 'No output'), (60, 'No output'), (64, 'No output'), (68, 'No output'), (70, 224), (74, 'No output'), (78, 'No output'), (82, 'No output'), (86, 'No output'), (90, 'No output'), (94, 'No output'), (98, 'No output'), (100, 224), (104, 'No output'), (108, 'No output'), (112, 'No output'), (116, 'No output'), (120, 'No output'), (122, 224), (126, 'No output'), (130, 'No output'), (134, 'No output'), (138, 'No output'), (142, 'No output'), (146, 'No output'), (150, 'No output'), (152, 224), (156, 'No output'), (160, 'No output'), (164, 'No output'), (168, 'No output'), (172, 'No output'), (176, 'No output'), (178, 224), (182, 'No output'), (186, 'No output'), (190, 'No output'), (194, 'No output'), (198, 'No output'), (200, 224), (

In [71]:
len(z[0])

61

In [78]:
print(program[48])

102
